In [42]:
import json 
import pandas as pd 



# Test model results 

In [43]:
test_model_results = []

with open("test_responses.jsonl",'r') as f : 
    for line in f: 
        if line.strip():
            data = json.loads(line)
            test_model_results.append(data)


In [44]:
df_test_model = pd.DataFrame(test_model_results)
df_test_model

,sample_id,test model provider,test model version,Encountered Problems,test model response
0,0,openai,gpt-3.5-turbo,False,Based on the information provided in the conte...
1,0,openai,gpt-4o,False,"Based on the context document, your risk facto..."
2,1,openai,gpt-3.5-turbo,False,1. Wüsthof - 14 degrees\n2. MAC - 15 degrees\n...
3,1,openai,gpt-4o,False,"Based on the provided information, here is a l..."


In [45]:
print(df_test_model['test model response'].iloc[0])

Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation. 

Cognitive engagement is important because it is thought to support the development of a "cognitive reserve," which may protect against brain cell damage caused by dementia. Engaging in activities that challenge and stimulate the brain throughout life may help reduce the risk of developing dementia.


In [46]:
print(df_test_model['test model response'].iloc[1])

Based on the context document, your risk factors for dementia include:

1. **Hearing Loss**: Having your ears blown out in the war may have led to hearing loss, which increases the risk of cognitive decline and dementia.

2. **Poor Diet**: Eating mostly garbage, which likely means an unhealthy diet high in saturated fat, sugar, and salt, can increase the risk of developing dementia.

3. **Depression**: Experiencing "a case of the sads pretty regular" suggests depression, which is associated with a higher risk of developing dementia.

Regarding cognitive engagement, the document suggests that actively using your brain throughout life can support the development of a "cognitive reserve." This cognitive reserve may offer some protection against brain cell damage caused by dementia. Therefore, engaging in activities that challenge your brain could potentially reduce your risk of developing dementia.


# Judges

In [47]:
judges_result = []

with open("judge_responses.jsonl",'r') as f : 
    for line in f:
        if line.strip():
            content = json.loads(line)
            judges_result.append(content)

In [48]:
df_judges = pd.DataFrame(judges_result)
df_judges

,judgeded_model,sample_id,judge model provider,judge model version,Encountered Problems,judge model response,verdict
0,openai:gpt-3.5-turbo,0,anthropic,claude-3-5-sonnet-20240620,False,Sentence 1: Based on the information provided ...,Accurate
1,openai:gpt-3.5-turbo,0,google_vertexai,gemini-1.5-pro,False,"```jsonl\n{""sentence"": ""Based on the informati...",None
2,openai:gpt-4o,0,anthropic,claude-3-5-sonnet-20240620,False,"Sentence 1: Based on the context document, you...",Accurate
3,openai:gpt-4o,0,google_vertexai,gemini-1.5-pro,False,"```json\n{""sentence"": ""Based on the context do...",Accurate
4,openai:gpt-3.5-turbo,1,anthropic,claude-3-5-sonnet-20240620,False,Sentence 1: Wüsthof - 14 degrees\nSentence 1 l...,Inaccurate
5,openai:gpt-3.5-turbo,1,google_vertexai,gemini-1.5-pro,False,"```json\n{""sentence"": ""1. Wüsthof - 14 degrees...",Accurate
6,openai:gpt-4o,1,anthropic,claude-3-5-sonnet-20240620,False,"Sentence 1: Based on the provided information,...",Accurate
7,openai:gpt-4o,1,google_vertexai,gemini-1.5-pro,False,"```json\n{""sentence"": ""Based on the provided i...",Accurate


In [49]:
print(len(df_test_model))
print(len(df_judges))

4
8


# Explore some samples 

In [50]:
print(df_judges['judge model response'].iloc[0])

Sentence 1: Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation.
Sentence 1 label: Accurate

Sentence 2: Cognitive engagement is important because it is thought to support the development of a "cognitive reserve," which may protect against brain cell damage caused by dementia.
Sentence 2 label: Accurate

Sentence 3: Engaging in activities that challenge and stimulate the brain throughout life may help reduce the risk of developing dementia.
Sentence 3 label: Accurate

Final Answer: Accurate


In [51]:
print(df_judges['judge model response'].iloc[1])

```jsonl
{"sentence": "Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation.", "label": "supported", "rationale": "All of the listed factors are mentioned as risk factors in the context.", "excerpt": "high blood pressure \n People who have consistent high blood pressure (hypertension) in mid-life (ages 45 to 65) are more likely to develop dementia compared to those with normal blood pressure.\n \n High blood pressure can increase the risk of developing dementia, particularly vascular dementia, because of its effect on the heart, the arteries, and blood circulation. \n \n Smoking \n The evidence is strong and consistent that smokers are at a higher risk of developing dementia vs. non-smokers or ex-smokers.\n \n It’s never too late to quit! Smoke

# Overall Accuracy 

In [52]:
TEST_MODEL = 'openai:gpt-3.5-turbo' # feel free to checkout accuracy for other models 

In [53]:
df_judges[df_judges['judgeded_model']==TEST_MODEL].groupby('judge model version')['verdict'].value_counts().unstack(fill_value=0)

verdict,Accurate,Inaccurate
judge model version,,
claude-3-5-sonnet-20240620,1,1
gemini-1.5-pro,1,0


In [54]:
df_filtered = df_judges[df_judges['judgeded_model'] == TEST_MODEL]
counts = df_filtered.groupby('judge model version')['verdict'].value_counts()
percentages = counts / counts.groupby(level=0).sum() * 100
result = percentages.unstack(fill_value=0)[["Accurate"]]

result

verdict,Accurate
judge model version,
claude-3-5-sonnet-20240620,50.0
gemini-1.5-pro,100.0
